In [1]:
import pandas as pd

# Load the CSV file
file_path_csv = 'v03-cnx-tambon-3_6705.csv'
data = pd.read_csv(file_path_csv)

# Display the first few rows of the dataset to understand its structure
print(data.head())


   ปีเดือน  รหัสจังหวัด       ชื่อจังหวัด  รหัสสำนักทะเบียน  \
0   6705.0         50.0  จังหวัดเชียงใหม่            5001.0   
1   6705.0         50.0  จังหวัดเชียงใหม่            5001.0   
2   6705.0         50.0  จังหวัดเชียงใหม่            5002.0   
3   6705.0         50.0  จังหวัดเชียงใหม่            5002.0   
4   6705.0         50.0  จังหวัดเชียงใหม่            5002.0   

      ชื่อสำนักทะเบียน  รหัสตำบล        ชื่อตำบล  ชาย < 1 ปี  หญิง < 1 ปี  \
0  อำเภอเมืองเชียงใหม่  500107.0   ตำบลช้างเผือก        15.0         17.0   
1  อำเภอเมืองเชียงใหม่  500116.0  ตำบลสันผีเสื้อ        56.0         48.0   
2          อำเภอจอมทอง  500203.0    ตำบลบ้านหลวง        65.0         83.0   
3          อำเภอจอมทอง  500204.0     ตำบลข่วงเปา        26.0         20.0   
4          อำเภอจอมทอง  500205.0    ตำบลสบเตี๊ยะ        28.0         31.0   

   ชาย 1 ปี  ...  หญิง 98 ปี  ชาย 99 ปี  หญิง 99 ปี  ชาย 100 ปี  หญิง 100 ปี  \
0      11.0  ...         0.0        0.0         0.0         0.0          0.0  

In [2]:
# List of age groups to combine
age_groups = [
    '< 1 ปี', '1 ปี', '2 ปี', '3 ปี', '4 ปี', '5 ปี', '6 ปี', '7 ปี', '8 ปี', '9 ปี',
    '10 ปี', '11 ปี', '12 ปี', '13 ปี', '14 ปี', '15 ปี', '16 ปี', '17 ปี', '18 ปี', '19 ปี',
    '20 ปี', '21 ปี', '22 ปี', '23 ปี', '24 ปี', '25 ปี', '26 ปี', '27 ปี', '28 ปี', '29 ปี',
    '30 ปี', '31 ปี', '32 ปี', '33 ปี', '34 ปี', '35 ปี', '36 ปี', '37 ปี', '38 ปี', '39 ปี',
    '40 ปี', '41 ปี', '42 ปี', '43 ปี', '44 ปี', '45 ปี', '46 ปี', '47 ปี', '48 ปี', '49 ปี',
    '50 ปี', '51 ปี', '52 ปี', '53 ปี', '54 ปี', '55 ปี', '56 ปี', '57 ปี', '58 ปี', '59 ปี',
    '60 ปี', '61 ปี', '62 ปี', '63 ปี', '64 ปี', '65 ปี', '66 ปี', '67 ปี', '68 ปี', '69 ปี',
    '70 ปี', '71 ปี', '72 ปี', '73 ปี', '74 ปี', '75 ปี', '76 ปี', '77 ปี', '78 ปี', '79 ปี',
    '80 ปี', '81 ปี', '82 ปี', '83 ปี', '84 ปี', '85 ปี', '86 ปี', '87 ปี', '88 ปี', '89 ปี',
    '90 ปี', '91 ปี', '92 ปี', '93 ปี', '94 ปี', '95 ปี', '96 ปี', '97 ปี', '98 ปี', '99 ปี',
    '100 ปี', '> 100 ปี'
]

# Convert age group columns to numeric, handle non-numeric values
for age in age_groups:
    male_col = f'ชาย {age}'
    female_col = f'หญิง {age}'
    combined_col = age
    
    if male_col in data.columns and female_col in data.columns:
        data[male_col] = pd.to_numeric(data[male_col], errors='coerce').fillna(0)
        data[female_col] = pd.to_numeric(data[female_col], errors='coerce').fillna(0)
        data[combined_col] = data[male_col] + data[female_col]

# Drop the original separate columns for men and women
cols_to_drop = [f'ชาย {age}' for age in age_groups if f'ชาย {age}' in data.columns] + \
               [f'หญิง {age}' for age in age_groups if f'หญิง {age}' in data.columns]
data_combined = data.drop(columns=cols_to_drop)

# Convert 'ผลรวมประชากรทั้งหมด' to numeric
data_combined['ผลรวมประชากรทั้งหมด'] = pd.to_numeric(data_combined['ผลรวมประชากรทั้งหมด'], errors='coerce').fillna(0)

# Calculate the total population from the combined age groups
data_combined['Calculated Total Population'] = data_combined[age_groups].sum(axis=1)

# Cross-check the calculated total with 'ผลรวมประชากรทั้งหมด'
data_combined['Difference'] = data_combined['ผลรวมประชากรทั้งหมด'] - data_combined['Calculated Total Population']

# Identify rows with discrepancies
discrepancies = data_combined[data_combined['Difference'] != 0]

# Save the combined data to a new CSV file
output_path_csv = 'combined_data.csv'
data_combined.to_csv(output_path_csv, index=False)

# Save discrepancies to a separate CSV file for review
discrepancies_output_path_csv = 'discrepancies.csv'
discrepancies.to_csv(discrepancies_output_path_csv, index=False)

# Print the results
print(f"Combined data has been saved to {output_path_csv}")
print(f"Discrepancies have been saved to {discrepancies_output_path_csv}")
print(f"Total discrepancies found: {len(discrepancies)}")

Combined data has been saved to combined_data.csv
Discrepancies have been saved to discrepancies.csv
Total discrepancies found: 253


/var/folders/4_/yy75nyfs02544ssmwvlgvz980000gn/T/ipykernel_6312/225058742.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[combined_col] = data[male_col] + data[female_col]
/var/folders/4_/yy75nyfs02544ssmwvlgvz980000gn/T/ipykernel_6312/225058742.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[combined_col] = data[male_col] + data[female_col]
/var/folders/4_/yy75nyfs02544ssmwvlgvz980000gn/T/ipykernel_6312/225058742.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of callin